# 妞妞
過年跟親戚打牌學會玩這個遊戲 <br>
希望能先藉由簡單的模擬來增加自己的贏面 <br>
這部分完成後希望能引入機器學習(GAN)之類的方法完善此模型 <br>

# import package

In [1]:
# import package
import itertools
import pandas as pd
import random
from collections import Counter

# 生成一副撲克牌

In [2]:
# def poker with rank & suit
suits = ['spade', 'heart', 'diamond', 'club']
ranks = ['A'] + [str(i) for i in range(2, 11)] + ['J', 'Q', 'K']

# generate a poker deck
deck = [(suit, rank) for suit in suits for rank in ranks]

# 轉換牌 & 排序大小

## 點數

In [3]:
# get card value
def card_value(card):
    # J, Q, K == 10
    # A == 1
    rank = card[1]
    return 10 if rank in ['J', 'Q', 'K'] else (1 if rank == 'A' else int(rank))

# get card rank
def get_card_rank(card):
    rank_order = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
                  'J': 11, 'Q': 12, 'K': 13, 'A': 1}
    return rank_order[card[1]]


## 花色

In [4]:
# get suit rank
def get_suit_rank(card):
    suit_order = {'spade': 4, 'heart': 3, 'diamond': 2, 'club': 1}
    return suit_order[card[0]]

# 計算 5 張牌的牌型

In [7]:
# caculate fix 5 card type
def calculate_niu(cards):
    # compare value rank then suit rank
    cards_value = [card_value(card) for card in cards]

    # sepcial card type
    if all(card[1] in ['J', 'Q', 'K'] for card in cards):
        return '五公'
    if all(v < 5 for v in cards_value) and sum(cards_value) == 10:
        return '五小牛'
    if sum(1 for card in cards if card[1] in ['J', 'Q', 'K']) == 4 and any(card[1] == '10' for card in cards):
        return '四花牛'

    # normal card type
    for combo in itertools.combinations(cards_value, 3):
        if sum(combo) % 10 == 0:
            remaining = [v for v in cards_value if v not in combo][:2]
            if sum(remaining) % 10 == 0:
                return '牛牛'
            else:
                return f'牛{sum(remaining) % 10}'

    # check J, Q, K
    return '有公無牛' if any(card[1] in ['J', 'Q', 'K'] for card in cards) else '無公無牛'


## 確認目前為止是否正確
1. 是否可以得到正確的牌型 <br>
測試出幾乎所有的情況

In [8]:
# hands
hand1 = [('spade', 'A'), ('heart', '2'), ('diamond', '3'), ('club', '2'), ('spade', '2')]
hand2 = [('spade', 'A'), ('heart', '2'), ('diamond', '3'), ('club', '2'), ('spade', '5')]
hand3 = [('spade', 'J'), ('heart', 'Q'), ('diamond', 'K'), ('club', 'J'), ('spade', 'Q')]
hand4 = [('spade', 'J'), ('heart', 'Q'), ('diamond', 'K'), ('club', 'J'), ('spade', '10')]
hand5 = [('spade', 'J'), ('heart', 'Q'), ('diamond', 'A'), ('club', '5'), ('spade', '8')]
hand6 = [('spade', 'A'), ('heart', '2'), ('diamond', '3'), ('club', '8'), ('spade', '3')]
hand7 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '10'), ('club', 'J'), ('spade', '10')]
hand8 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '10'), ('club', 'J'), ('spade', '9')]
hand9 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '10'), ('club', 'J'), ('spade', '8')]
hand10 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '10'), ('club', 'J'), ('spade', '7')]
hand11 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '10'), ('club', 'J'), ('spade', '5')]

# result
print(calculate_niu(hand1))
print(calculate_niu(hand2))
print(calculate_niu(hand3))
print(calculate_niu(hand4))
print(calculate_niu(hand5))
print(calculate_niu(hand6))
print(calculate_niu(hand7))
print(calculate_niu(hand8))
print(calculate_niu(hand9))
print(calculate_niu(hand10))
print(calculate_niu(hand11))


五小牛
牛1
五公
四花牛
有公無牛
無公無牛
牛牛
牛9
牛8
牛7
牛5


# 模擬牌型出現機率

In [25]:
# def simulate card type rate
def simulate_cardtype_rate(hand, num_simulations=500000):
    # generate poker deck
    suits = ['spade', 'heart', 'diamond', 'club']
    ranks = [str(i) for i in range(2, 11)] + ['J', 'Q', 'K', 'A']
    deck = [(suit, rank) for suit in suits for rank in ranks]

    # remove cards already in hand
    for card in hand:
        deck.remove(card)

    results = Counter()
    for _ in range(num_simulations):
        # random choose a card
        remaining_card = random.choice(deck)
        final_hand = hand + [remaining_card]
        # only want to know card type
        card_type = calculate_niu(final_hand)
        results[card_type] += 1

    total = sum(results.values())
    probabilities = {k: v / total for k, v in results.items()}
    sorted_probabilities = dict(sorted(probabilities.items(), key=lambda item: item[1], reverse=True))
    return sorted_probabilities


## 確認機率情況

In [26]:
hand1 = [('spade', 'J'), ('heart', 'Q'), ('diamond', '3'), ('club', '2')]
simulate_cardtype_rate(hand1)

{'有公無牛': 0.45794,
 '牛5': 0.291704,
 '牛牛': 0.08359,
 '牛3': 0.083494,
 '牛2': 0.083272}

# 如果牌型一樣時會需要比較最高張
比較手牌最高張

In [ ]:
# only compare the highest card
# return who win
def compare_highest_card(player_hand, banker_hand):
    player_hand_sorted = sorted(player_hand, key=lambda x: (get_card_rank(x), get_suit_rank(x)), reverse=True)
    banker_hand_sorted = sorted(banker_hand, key=lambda x: (get_card_rank(x), get_suit_rank(x)), reverse=True)
    
    player_max_card = player_hand_sorted[0]
    banker_max_card = banker_hand_sorted[0]

    if get_card_rank(player_max_card) > get_card_rank(banker_max_card):
        return "player"
    elif get_card_rank(player_max_card) < get_card_rank(banker_max_card):
        return "banker"
    else:
        if get_suit_rank(player_max_card) > get_suit_rank(banker_max_card):
            return "player"
        else:
            return "banker"

# 結算金額

In [ ]:
# caculate payout
def calculate_payout(player_hand, banker_hand):
    """根據玩家和莊家的手牌計算賠率"""

    if len(set(player_hand)) != len(player_hand):
        return "錯誤：玩家手牌有重複的牌！"
    if len(set(banker_hand)) != len(banker_hand):
        return "錯誤：莊家手牌有重複的牌！"
    combined_hand = player_hand + banker_hand
    if len(set(combined_hand)) != len(combined_hand):
        return "錯誤：玩家與莊家有相同的牌！"


    # def hand's rank
    rank_order = {
        '五小牛': 15, '五公': 14, '四花牛': 13, '牛牛': 12, '牛9': 11, '牛8': 10,
        '牛7': 9, '牛6': 8, '牛5': 7, '牛4': 6, '牛3': 5, '牛2': 4, '牛1': 3,
        '有公無牛': 2, '無公無牛': 1
    }

    # def payout of hand
    rank_mult = {
        '五小牛': 6, '五公': 5, '四花牛': 4, '牛牛': 3, '牛9': 2, '牛8': 2,
        '牛7': 2, '牛6': 1, '牛5': 1, '牛4': 1, '牛3': 1, '牛2': 1, '牛1': 1,
        '有公無牛': 1, '無公無牛': 2
    }

    # get player & banker hand
    player_type = calculate_niu(player_hand)
    banker_type = calculate_niu(banker_hand)

    player_rank = rank_order[player_type]
    banker_rank = rank_order[banker_type]
    player_mult = rank_mult[player_type]
    banker_mult = rank_mult[banker_type]

    # compare hand rank
    if player_rank > banker_rank:
        multiplier = player_mult if banker_type != '無公無牛' else player_mult * 2
        return f"玩家贏，賠率為 {multiplier} 倍"
    
    elif player_rank < banker_rank:
        multiplier = -banker_mult if player_type != '無公無牛' else -banker_mult * 2
        return f"玩家輸，賠率為 {multiplier} 倍"

    # if same hand, compare max card
    player_max_card = max(player_hand, key=lambda x: (get_card_rank(x), get_suit_rank(x)))
    banker_max_card = max(banker_hand, key=lambda x: (get_card_rank(x), get_suit_rank(x)))

    if get_card_rank(player_max_card) > get_card_rank(banker_max_card):
        return f"玩家贏，賠率為 {player_mult} 倍"
    elif get_card_rank(player_max_card) < get_card_rank(banker_max_card):
        return f"玩家輸，賠率為 {-banker_mult} 倍"

    # if same card num, compare suit rank
    if get_suit_rank(player_max_card) > get_suit_rank(banker_max_card):
        return f"玩家贏，賠率為 {player_mult} 倍"
    else:
        return f"玩家輸，賠率為 {-banker_mult} 倍"


## 確認其中的一些情況

In [32]:
# banker wins
## normal
player_hand = [('heart', '9'), ('diamond', 'J'), ('club', '3'), ('spade', '6'), ('heart', '2')]
banker_hand = [('spade', '10'), ('club', 'J'), ('heart', '4'), ('diamond', '6'), ('diamond', '2')]
print(calculate_payout(player_hand, banker_hand))

## banker have double
player_hand = [('heart', '9'), ('diamond', 'J'), ('club', '3'), ('spade', '6'), ('heart', '2')]
banker_hand = [('spade', '10'), ('club', 'J'), ('heart', '4'), ('diamond', '6'), ('diamond', '7')]
print(calculate_payout(player_hand, banker_hand))

## banker have niuniu
player_hand = [('heart', '9'), ('diamond', 'J'), ('club', '3'), ('spade', '6'), ('heart', '2')]
banker_hand = [('spade', '10'), ('club', 'J'), ('heart', '4'), ('diamond', '6'), ('diamond', '4')]
print(calculate_payout(player_hand, banker_hand))

## player have nothing
player_hand = [('heart', '9'), ('diamond', '3'), ('club', '3'), ('spade', '6'), ('heart', '2')]
banker_hand = [('spade', '10'), ('club', 'J'), ('heart', '4'), ('diamond', '6'), ('diamond', '7')]
print(calculate_payout(player_hand, banker_hand))

# banker wins
## normal
player_hand = [('spade', '10'), ('club', 'J'), ('heart', '4'), ('diamond', '6'), ('diamond', '2')]
banker_hand = [('heart', '9'), ('diamond', 'J'), ('club', '3'), ('spade', '6'), ('heart', '2')]
print(calculate_payout(player_hand, banker_hand))

# error
# for player have same card, raise error
player_hand = [('heart', '9'), ('diamond', 'J'), ('club', '3'), ('spade', '6'), ('heart', '9')]
print(calculate_payout(player_hand, banker_hand))

# for banker have same card, raise error
banker_hand = [('spade', '10'), ('club', 'J'), ('heart', '4'), ('diamond', '6'), ('spade', '10')]
print(calculate_payout(player_hand, banker_hand))

# for banker & player have same card, raise error
player_hand = [('heart', '9'), ('diamond', 'J'), ('club', '3'), ('spade', '6'), ('heart', '2')]
banker_hand = [('spade', '10'), ('club', 'J'), ('heart', '4'), ('diamond', '6'), ('heart', '9')]
print(calculate_payout(player_hand, banker_hand))


玩家輸，賠率為 -1 倍
玩家輸，賠率為 -2 倍
玩家輸，賠率為 -3 倍
玩家輸，賠率為 -4 倍
玩家贏，賠率為 1 倍
錯誤：玩家手牌有重複的牌！
錯誤：玩家手牌有重複的牌！
錯誤：玩家與莊家有相同的牌！


# 進行預估(單純自己手牌)
只預測是否下大注

# 實際操作